In [1]:
import pandas as pd
import sqlite3
from datetime import datetime

In [2]:
df= pd.read_csv("synthetic_client_queries.csv")

In [3]:
df

,query_id,client_email,client_mobile,query_heading,query_description,status,date_raised,date_closed
0,Q0001,joshuaoneill@example.net,6638686049,Bug Report,Tab focus jumps incorrectly.,Closed,2025-02-26,2025-02-27
1,Q0002,edward91@example.net,6369365581,Bug Report,Form validation not working properly.,Closed,2025-07-22,2025-08-01
2,Q0003,cookmason@example.org,2686615523,Account Suspension,Need help lifting restrictions.,Closed,2025-06-18,2025-06-19
3,Q0004,wtaylor@example.com,4753393384,Data Export,Need monthly data dump in CSV.,Closed,2025-02-25,2025-03-07
4,Q0005,rmoore@example.net,8523755943,UI Feedback,Icons not intuitive.,Closed,2025-06-19,2025-06-20
...,...,...,...,...,...,...,...,...
5195,Q5196,fhenry@example.org,2758293072,Payment Failure,EMI option not available anymore.,Closed,2025-05-03,2025-05-09
5196,Q5197,robert82@example.net,2400038786,Technical Support,Freezing during screen load.,Closed,2025-05-17,2025-05-20
5197,Q5198,mclaughlinmichael@example.net,312901852,Login Issue,Forgot my password and reset isn’t working.,Closed,2025-06-25,2025-07-04
5198,Q5199,vaughnkristen@example.org,795334441,UI Feedback,Graphs too compressed on mobile view.,Closed,2025-07-15,2025-07-24


In [4]:
# Drop duplicate rows
df.drop_duplicates(inplace=True)

In [5]:
# Drop rows with missing essential values
required_cols = ['query_id', 'client_email', 'query_heading']
df.dropna(subset=required_cols, inplace=True)

In [6]:
# Convert date columns to datetime
for col in ['date_raised', 'date_closed']:
    if col in df.columns:
        df[col] = pd.to_datetime(df[col], errors='coerce')

In [7]:
df['client_mobile'] = df['client_mobile'].astype(str)

In [8]:
df.reset_index(drop=True, inplace=True)

In [9]:
db_path = "queries_database.db"

# Connect to or create a new database
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

In [10]:
cursor.execute("""
CREATE TABLE IF NOT EXISTS queries (
    query_id TEXT PRIMARY KEY,
    client_email TEXT,
    client_mobile TEXT,
    query_heading TEXT,
    query_description TEXT,
    status TEXT,
    date_raised TEXT,
    date_closed TEXT
)
""")

conn.commit()
print("✅ Database and table created successfully.")

✅ Database and table created successfully.


In [11]:
existing_ids = pd.read_sql("SELECT query_id FROM queries", conn)['query_id'].tolist()

In [12]:
new_rows = df[~df['query_id'].isin(existing_ids)]

if not new_rows.empty:
    new_rows.to_sql('queries', conn, if_exists='append', index=False)
    conn.commit()
    print(f"✅ {len(new_rows)} new rows inserted into the database.")
else:
    print("ℹ️ No new rows to insert. Database is already up to date.")


✅ 5200 new rows inserted into the database.


In [13]:
stored_df = pd.read_sql("SELECT * FROM queries", conn)
print(f"\n📊 Total rows in database: {len(stored_df)}")
print(stored_df.head())


📊 Total rows in database: 5200
  query_id              client_email client_mobile       query_heading  \
0    Q0001  joshuaoneill@example.net    6638686049          Bug Report   
1    Q0002      edward91@example.net    6369365581          Bug Report   
2    Q0003     cookmason@example.org    2686615523  Account Suspension   
3    Q0004       wtaylor@example.com    4753393384         Data Export   
4    Q0005        rmoore@example.net    8523755943         UI Feedback   

                       query_description  status          date_raised  \
0           Tab focus jumps incorrectly.  Closed  2025-02-26 00:00:00   
1  Form validation not working properly.  Closed  2025-07-22 00:00:00   
2        Need help lifting restrictions.  Closed  2025-06-18 00:00:00   
3         Need monthly data dump in CSV.  Closed  2025-02-25 00:00:00   
4                   Icons not intuitive.  Closed  2025-06-19 00:00:00   

           date_closed  
0  2025-02-27 00:00:00  
1  2025-08-01 00:00:00  
2  2025-0